In [ ]:
import toml
import re
import urllib
import pygsheets
import benchlingapi
import requests_html

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
from paulssonlab.api.util import base_url
import paulssonlab.cloning.workflow as workflow

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
session = benchlingapi.Session(config["benchling"]["api_key"])

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
col = workflow.get_strain_collection_sheets(gc.drive.service, "LIB")
col

In [ ]:
strain_sheet = gc.open_by_key(col["strains"]).worksheet()
plasmid_sheet = gc.open_by_key(col["plasmids"]).worksheet()

# 3G/JUMP/Densmore

In [ ]:
threeg_kit = "https://www.addgene.org/1000000161/"
marionette_kit = "https://www.addgene.org/1000000137/"
jump_plasmids = [
    "https://www.addgene.org/126956/",
    "https://www.addgene.org/126959/",
    "https://www.addgene.org/126960/",
    "https://www.addgene.org/126961/",
    "https://www.addgene.org/126962/",
    "https://www.addgene.org/126963/",
    "https://www.addgene.org/126964/",
    "https://www.addgene.org/126965/",
    "https://www.addgene.org/126966/",
    "https://www.addgene.org/126967/",
    "https://www.addgene.org/126973/",
    "https://www.addgene.org/126974/",
    "https://www.addgene.org/126975/",
    "https://www.addgene.org/126976/",
    "https://www.addgene.org/126991/",
    "https://www.addgene.org/126996/",
    "https://www.addgene.org/127015/",
    "https://www.addgene.org/127047/",
    "https://www.addgene.org/127051/",
    "https://www.addgene.org/127025/",
    "https://www.addgene.org/127000/",
    "https://www.addgene.org/126983/",
]
jump_plasmids = sorted(jump_plasmids)
densmore_kit = "https://www.addgene.org/1000000059/"
densmore_wells = [
    "A1",
    "A5",
    "A9",
    "B1",
    "B5",
    "B9",
    "C1",
    "C5",
    "C9",
    "D1",
    "D5",
    "D6",
    "D7",
    "D8",
    "D9",
    "D10",
    "D11",
    "D12",
    "E1",
    "E2",
    "E3",
    "E4",
    "E5",
    "E6",
    "E7",
]

# Densmore renaming

In [ ]:
for sheet in (strain_sheet, plasmid_sheet):
    rows = sheet.get_all_records()
    # sheet.unlink()
    col_idx = list(rows[0].keys()).index("Aliases*") + 1
    for idx, row in enumerate(rows):
        if base_url(densmore_kit) in row["Source*"]:
            new_aliases = re.sub(
                r"([^()]*)\s\(([^()]*)\)([^()]*)", r"\1\3,\2\3", row["Aliases*"]
            )
            sheet.update_value((idx + 2, col_idx), new_aliases, parse=False)
    # sheet.link() # TODO: this gives 500 error, not sure why

# Ingest parts

In [ ]:
rows = plasmid_sheet.get_all_records()

In [ ]:
rows[0]

In [ ]:
def import_threeg_part(plasmid):
    pass


def import_densmore_part(plasmid):
    pass


def import_jump_part(plasmid):
    pass


# accept extra columns via overrides={"Tags": "foo"}
# pass through tags from plasmid


def get_part():
    part = {}
    part["Name*"] = ""
    part["Author*"] = ""
    part["Date*"] = ""
    part["Tags"] = ""
    part["Plasmid/Oligos (Cutter)*"] = ""
    part["Author*"] = ""
    part["Date*"] = ""
    part["Upstream overhang*"] = ""
    part["Downstream overhang*"] = ""
    part["Sequence*"] = ""
    part["Organism/codon usage*"] = ""
    part["Description"] = ""
    return part


part_rules = [
    (
        lambda x: (base_url(threeg_kit) in x["Source*"]) and (x["Aliases*"][0] != "V"),
        import_threeg_part,
    ),
    (lambda x: base_url(densmore_kit) in x["Source*"], import_densmore_part),
    (
        lambda x: (any(base_url(j) in x["Source*"] for j in jump_plasmids))
        and ("(Empty Backbone)" not in x["Description"]),
        import_jump_part,
    ),
]

In [ ]:
for row in rows:
    for predicate, rule in part_rules:
        if predicate(row):
            print(rule, row["Aliases*"])
            break